In [1]:
import pandas as pd
import numpy as np
from loguru import logger
from utils import open_record_directory, open_result_directory
from nltk.translate import meteor
from nltk import word_tokenize
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


/home/ubuntu/miniforge3/envs/multimodal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def getMeteorScore(filename):
    df = pd.read_csv(filename)
    df_clean = df.dropna()
    
    scores = [meteor([word_tokenize(x['fut_summary'])], word_tokenize(x['pred_summary'])) for idx, x in df_clean.iterrows()]
    # print(scores)
    mean_score=np.mean(scores)
    # Display the dataframe with the METEOR scores
    return mean_score

In [27]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

def getCosineSimilarity(filename):
    df = pd.read_csv(filename)
    df.replace("Not available", np.nan, inplace=True)
    df_clean = df.dropna()
    
    ground_truth = df_clean['ground_truth'].tolist()
    zero_shot = df_clean['zero_shot'].tolist()
    fine_tuned = df_clean['fine_tuned'].tolist()
    # print(scores)
    model = SentenceTransformer('thenlper/gte-base')
    ground_truth_embeddings = model.encode(ground_truth)
    zero_shot_embeddings = model.encode(zero_shot)
    fine_tuned_embeddings = model.encode(fine_tuned)

    zs_cos_sims = [cos_sim(x, y) for x, y in zip(ground_truth_embeddings, zero_shot_embeddings)]
    ft_cos_sims = [cos_sim(x, y) for x, y in zip(ground_truth_embeddings, fine_tuned_embeddings)]
    # Display the dataframe with the METEOR scores
    return np.mean(zs_cos_sims), np.mean(ft_cos_sims)


In [7]:
mean_score = getMeteorScore(filename="/home/ubuntu/multimodal/Predictions_and_attempts/Gas/1_week/text-text/zeroshot/llama7b_output_validation_all.csv")
print(mean_score)
# zs_mean_score, ft_mean_score = getCosineSimilarity(filename="/home/ubuntu/multimodal/Predictions_and_attempts/pred_combined.csv")
# print("zs_mean_score: ", zs_mean_score)
# print("ft_mean_score: ", ft_mean_score)


0.445649965934562
